In [2]:
from typing import Optional

from tqdm import trange
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.experimental import disable_dynamic_shapes
from torch_geometric.nn.aggr import Aggregation
from torch_geometric.utils import coalesce

import pathpyG as pp

# DeBruijn Transformations using GNNs

In [3]:
class ConcatAggregation(Aggregation):
    
    def __init__(self):
        super().__init__()

    # Not sure how this aggregation works, only that it does
    # Inspired by the LSTMAggregation implementation in PyG:
    # https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/aggr/lstm.html
    @disable_dynamic_shapes(required_args=['dim_size', 'max_num_elements'])
    def forward(
        self,
        x: Tensor,
        index: Optional[Tensor] = None,
        ptr: Optional[Tensor] = None,
        dim_size: Optional[int] = None,
        dim: int = -2,
        max_num_elements: Optional[int] = None,
    ) -> Tensor:

        # Concetenate all messages with padding value -1
        x, _ = self.to_dense_batch(x, index, ptr, dim_size, dim, max_num_elements=max_num_elements, fill_value=-1)
        return x


class DeBruijnTransform(MessagePassing):
    # freq_aggr is a string specifying how we should count each path in the network
    # This can either be `propagation` where every path is counted once
    # or `diffusion` where every path is counted depending on the number of outgoing edges
    # of the source node.
    def __init__(self, freq_aggr: str = "propagation"):
        super().__init__(aggr=ConcatAggregation(), flow="target_to_source")
        self.freq_aggr = freq_aggr

    def forward(self, node_idx, edge_index, edge_attr=None):
        # Sort edge_index because otherwise propagate will not work in combination with the ConcatAggregation
        edge_index = coalesce(edge_index, sort_by_row=True)
        # Set the dimension along which the node feature tensor is expected
        # This is the default value, but we need to set it explicitly here
        # because we change it later
        self.node_dim = -2
        # Update the node idx by passing the messages and aggregating them
        # In the message function, we concatenate the node idx of the source node
        # with the node idx of the target node
        # In the aggregation function, we concatenate all messages from the neighbors
        # The resulting feature for every node is a tensor of shape (max_degree, 2)
        # where max_degree is the maximum degree of the graph
        # If a node has less neighbors than max_degree, the remaining entries are filled with -1
        node_idx_set_higher_order = self.propagate(edge_index, node_idx=node_idx)
        # Since our node features changed from shape (N, 1) to (N, max_degree, 2)
        # we need to set the node_dim to -3
        self.node_dim = -3
        # We use the function that is used to update node features to create the higher order edges
        edge_index_higher_order, edge_attr_higher_order = self.edge_updater(
            edge_index, 
            node_idx=node_idx_set_higher_order,
            edge_attr=edge_attr
            )

        if edge_attr_higher_order is not None:
            return edge_index_higher_order, edge_attr_higher_order
        return edge_index_higher_order

    def message(self, node_idx_i, node_idx_j):
        # Concatenate the node idx of the source node with the node idx of the target node
        # The shape changes from (N, k) to (N, k+1) where k is the order of the nodes before
        return torch.cat([node_idx_i, node_idx_j[:, -1:]], dim=-1)
    
    def edge_update(self, node_idx_i, node_idx_j, edge_attr=None) -> tuple[Tensor, Optional[Tensor]]:
        # We take the higher order node idx sets that have been created for each node adjacent
        # to the edge (node_idx_i, node_idx_j) and repeat each node idx across different dimensions
        # so that we can compare them with each other
        #
        # Example:
        #
        #   node_idx_i = [[0, 3], [1, 3], [2, 3]]
        #   node_idx_j = [[3, 4], [2, 4], [-1, -1]]
        #
        #   strided_node_idx_i = [[
        #                           [0, 3],
        #                           [1, 3], 
        #                           [2, 3]
        #                         ],
        #                         [
        #                           [0, 3],
        #                           [1, 3],
        #                           [2, 3]
        #                         ],
        #                         [
        #                           [0, 3],
        #                           [1, 3],
        #                           [2, 3]
        #                         ]]
        #   strided_node_idx_j = [[
        #                           [3, 4],
        #                           [3, 4],
        #                           [3, 4]
        #                         ],
        #                         [
        #                           [2, 4],
        #                           [2, 4],
        #                           [2, 4]
        #                         ],
        #                         [
        #                           [-1, -1],
        #                           [-1, -1],
        #                           [-1, -1]
        #                         ]]
        strided_node_idx_i = node_idx_i.unsqueeze(1).expand(-1, node_idx_j.size(1), -1, -1)
        strided_node_idx_j = node_idx_j.unsqueeze(2).expand(-1, -1, node_idx_i.size(1), -1)
        # Only create an higher order edge if the target node idx of the first edge is equal to
        # the source node idx of the second edge
        edge_mask = (strided_node_idx_i[:, :, :, 1:] == strided_node_idx_j[:, :, :, :-1]).all(dim=-1)
        # Also, we need to remove the -1 padding values
        padd_mask = (
            (strided_node_idx_i[:, :, :] != -1).all(dim=-1) &
            (strided_node_idx_j[:, :, :] != -1).all(dim=-1)
        )
        # For the above, the following mask is:
        #
        #   mask = [[True, True, True],
        #           [False, False, False],
        #           [False, False, False]]
        mask = (edge_mask & padd_mask)
        # Concetenate the remaining higher order edges to create a new edge index
        higher_order_edges = torch.cat([strided_node_idx_i[mask].unsqueeze(0), strided_node_idx_j[mask].unsqueeze(0)], dim=0)
        
        if edge_attr is not None:
            # If edge attributes are given, we need to fit the shape to apply the same mask
            strided_edge_attr = edge_attr.unsqueeze(1).unsqueeze(1).expand(-1, node_idx_j.size(1), node_idx_i.size(1))
            # Apply the mask and use some way to combine the edge attributes of source and target node
            # For now, we just take the edge attribute of the source node
            if self.freq_aggr == "propagation":
                higher_order_edge_attr = strided_edge_attr[mask]
            elif self.freq_aggr == "diffusion":
                higher_order_edge_attr = strided_edge_attr[mask] / mask.sum(dim=(1,2), keepdim=True).expand(-1, node_idx_j.size(1), node_idx_i.size(1))[mask]
            else:
                raise ValueError(f"Unknown frequency aggregation method {self.freq_aggr}")
            return higher_order_edges, higher_order_edge_attr
        return higher_order_edges, None

## Toy Example

In [3]:
edge_index = torch.tensor([[0, 0, 1, 1, 3, 4, 1, 6, 5],
                           [1, 3, 2, 3, 4, 5, 6, 5, 7]])
node_idx = torch.arange(edge_index.max() + 1).reshape(-1, 1)
edge_index_2_fast = DeBruijnTransform()(node_idx, edge_index)
print(edge_index_2_fast)

tensor([[[0, 1],
         [0, 1],
         [0, 1],
         [0, 3],
         [1, 3],
         [1, 6],
         [3, 4],
         [4, 5],
         [6, 5]],

        [[1, 2],
         [1, 3],
         [1, 6],
         [3, 4],
         [3, 4],
         [6, 5],
         [4, 5],
         [5, 7],
         [5, 7]]])


In [4]:
edge_index_2 = pp.DAGData.lift_order_dag(edge_index.unsqueeze(-1))
print(edge_index_2)

tensor([[[0, 1],
         [0, 1],
         [0, 1],
         [0, 3],
         [1, 3],
         [3, 4],
         [4, 5],
         [6, 5],
         [1, 6]],

        [[1, 2],
         [1, 3],
         [1, 6],
         [3, 4],
         [3, 4],
         [4, 5],
         [5, 7],
         [5, 7],
         [6, 5]]])


In [5]:
print((edge_index_2.sort(dim=1)[0] == edge_index_2_fast.sort(dim=1)[0]).all())

tensor(True)


### With Edge Weights

Depending on how you count each walk, you will get different statistics. We can choose the aggregation via `freq_aggr` to be either "propagation", i.e. each walk counts with its weight, or "diffusion" i.e. each walk is counted with the probability of a random walker starting at the first node to end up in the last. 

In [6]:
edge_index = torch.tensor([[0, 0, 1, 1, 3, 4, 1, 6, 5],
                           [1, 3, 2, 3, 4, 5, 6, 5, 7]])
node_idx = torch.arange(edge_index.max() + 1).reshape(-1, 1)
edge_attr = torch.tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.float32)
edge_index_2_fast, edge_attr_2 = DeBruijnTransform(freq_aggr="diffusion")(node_idx, edge_index, edge_attr)
print(edge_index_2_fast)

tensor([[[0, 1],
         [0, 1],
         [0, 1],
         [0, 3],
         [1, 3],
         [1, 6],
         [3, 4],
         [4, 5],
         [6, 5]],

        [[1, 2],
         [1, 3],
         [1, 6],
         [3, 4],
         [3, 4],
         [6, 5],
         [4, 5],
         [5, 7],
         [5, 7]]])


In [7]:
print(edge_attr_2)

tensor([0.3333, 0.3333, 0.3333, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


### 3rd order
Currently, the transformation works only with a standard edge index. The good thing is, you can still pass in the higher_order node_idx and then the output is directly a third order edge index.

In [8]:
# Transform the edge index since the DeBruijnTransform only works for the normal edge index
edges_2 = edge_index_2_fast.reshape(-1, 2)
uniques, inverse_idx = edges_2.unique(dim=0, return_inverse=True)
transformed_edge_index_2_fast = inverse_idx.reshape(2, -1)

In [9]:
edge_index_3_fast = DeBruijnTransform()(uniques, transformed_edge_index_2_fast)
print(edge_index_3_fast)

tensor([[[0, 1, 3],
         [0, 1, 6],
         [0, 3, 4],
         [1, 3, 4],
         [1, 6, 5],
         [3, 4, 5]],

        [[1, 3, 4],
         [1, 6, 5],
         [3, 4, 5],
         [3, 4, 5],
         [6, 5, 7],
         [4, 5, 7]]])


In [10]:
edge_index_3 = pp.DAGData.lift_order_dag(edge_index_2)
print(edge_index_3)

tensor([[[0, 1, 3],
         [0, 1, 6],
         [0, 3, 4],
         [1, 3, 4],
         [3, 4, 5],
         [1, 6, 5]],

        [[1, 3, 4],
         [1, 6, 5],
         [3, 4, 5],
         [3, 4, 5],
         [4, 5, 7],
         [6, 5, 7]]])


# Exponentionally Large DAG

In [11]:
layers = 5
branches = 15

edges = []
prev_layer_nodes = [0]
j = 1
for _ in trange(layers):
    layer_nodes = []
    for node in prev_layer_nodes:
        for _ in range(branches):
            layer_nodes.append(j)
            edges.append((f"{node}", f"{j}"))
            j+=1
    prev_layer_nodes = layer_nodes

dag = pp.Graph.from_edge_list(edges)
dag_edge_index = dag.data.edge_index.unsqueeze(-1)

node_idx = torch.arange(dag_edge_index.max().item() + 1).unsqueeze(-1)
node_idx_gpu = node_idx.cuda()
dag_edge_index_gpu = dag.data.edge_index.cuda()

100%|██████████| 5/5 [00:00<00:00, 18.76it/s]

### Current implementation

In [12]:
%timeit pp.DAGData.lift_order_dag(dag_edge_index)

1min 9s ± 3.52 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Message Passing based implementation (CPU)

In [13]:
%timeit DeBruijnTransform()(node_idx, dag.data.edge_index)

694 ms ± 15.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Message Passing based implementation (GPU)

In [14]:
%timeit DeBruijnTransform()(node_idx_gpu, dag_edge_index_gpu)

81.6 ms ± 2.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
# print((dag_edge_index_order_2.sort(dim=1)[0] == dag_edge_index_order_2_fast.sort(dim=1)[0]).all())

## Many Walks

In [4]:
n_walks = 10000
walk_length = 1000

walks = [list(range(walk_length)) for _ in range(n_walks)]
orig_walk = pp.WalkData()
for walk in walks:
    orig_walk.add_walk_seq(walk)

path_list = list(orig_walk.paths.values())
path_freq_tensor = torch.tensor(list(orig_walk.path_freq.values()))
mapping = pp.IndexMap()
nested_walk = pp.WalkDataNested(path_list, path_freq=path_freq_tensor, mapping=mapping)

/workspaces/pathpyG/src/pathpyG/core/WalkDataNested.py:56: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  self.paths = nested_tensor(paths, dtype=torch.long)


### Original Walk Implementation

In [5]:
%timeit orig_walk.edge_index_k_weighted(2)

37.5 s ± 7.25 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Nested Tensor Implementation (CPU)

In [6]:
pp.config['torch']["device"] = "cpu"
%timeit nested_walk.edge_index_k_weighted(2)

466 ms ± 20.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Nested Tensor (GPU)

In [7]:
pp.config['torch']["device"] = "cuda"
cuda_path_list = [path.cuda() for path in path_list]
cuda_path_freq = path_freq_tensor.cuda()
cuda_nested_walk = pp.WalkDataNested(cuda_path_list, path_freq=cuda_path_freq, mapping=mapping)
%timeit cuda_nested_walk.edge_index_k_weighted(2)

408 ms ± 27.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Message Passing Implementation (CPU + GPU)

In [8]:
# We create a list of Data objects where each Data object contains the edge index of a path (could also be a DAG in theory)
data_list = [Data(edge_index=path.long(), num_nodes=walk_length) for path in path_list]
# We use a dataloader from PyG to combine all the edge indices into a single graph with multiple disjoint subgraphs
# If two paths share a node, the node is duplicated in the resulting graph and the new higher order edges need to be aggregated afterwards
# Note that due to the `batch_size` parameter, we can also do computations on a set of paths that are too large to fit into memory at once
walk_graph = next(iter(DataLoader(data_list, batch_size=n_walks)))
edge_index = walk_graph.edge_index
node_idx = torch.arange(edge_index.max() + 1).unsqueeze(-1)

The following measures the time to do the De Bruijn graph transformation for the edge index that contains all paths as disjunct subgraphs. Since the aggregations afterwards are omitted, the runtimes are not exactly comparable to the above. See the next section (With Weights and the Aggregation) for a full `edge_index_k_weighted` transformation.

In [9]:
%timeit DeBruijnTransform()(node_idx, edge_index)

1.06 s ± 64.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
cuda_edge_index = edge_index.cuda()
cuda_node_idx = node_idx.cuda()
%timeit DeBruijnTransform()(cuda_node_idx, cuda_edge_index)

59.8 ms ± 5.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### With Weights and the Aggregation

In [11]:
def edge_index_k_weighted(path_list, path_freq, aggregation="propagation", device="cuda"):
    data_list = [
        Data(
            edge_index=path.long(), 
            num_nodes=walk_length,
            edge_attr=torch.ones(path.size(1), dtype=torch.float32) * path_freq[i],
            node_idx=torch.arange(walk_length).unsqueeze(-1)
        ) for i, path in enumerate(path_list)
        ]
    walk_graph = next(iter(DataLoader(data_list, batch_size=n_walks, follow_batch=["node_idx"]))).to(device)
    edge_index = walk_graph.edge_index
    edge_attr = walk_graph.edge_attr
    node_idx = torch.arange(edge_index.max() + 1, device=device).unsqueeze(-1)
    edge_index_2, edge_attr_2 = DeBruijnTransform(aggregation)(node_idx, edge_index, edge_attr)
    orig_edge_index_2 = walk_graph.node_idx.squeeze()[edge_index_2]
    unique_edge_index_2, inverse_idx = orig_edge_index_2.unique(dim=1, return_inverse=True)
    edge_attr_2 = torch.zeros(unique_edge_index_2.size(1), device=device).index_add(0, inverse_idx, edge_attr_2)
    return unique_edge_index_2, edge_attr_2

In [12]:
%timeit edge_index_k_weighted(path_list, path_freq_tensor, device="cuda")

774 ms ± 48.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Index Translations using torch Tensors

In [13]:
edge_index = torch.tensor([[0,1,2], [1,2,3]])
# The indices should be mapped as follows: {0: 1, 1: 0, 2: 3, 3: 2}
mapping = torch.tensor([1, 0, 3, 2])
edge_index_mapped = mapping[edge_index]
print(edge_index_mapped)

tensor([[1, 0, 3],
        [0, 3, 2]])


In [19]:
import numpy as np
string_mapping = np.array(["a->b", "b->c", "c->d", "d->e"])
edge_index_string_mapped = string_mapping[edge_index]
edge_index_string_mapped

array([['a->b', 'b->c', 'c->d'],
       ['b->c', 'c->d', 'd->e']], dtype='<U4')